In [44]:
post_code_df = spark.read.format('csv').option("header", "true").load("/mnt/c/users/lohith/Downloads/National_Statistics_Postcode_Lookup_UK_Coordinates.csv")


In [49]:
post_code_df.transform(with_std_column_names()).printSchema()

root
 |-- postcode_1: string (nullable = true)
 |-- postcode_2: string (nullable = true)
 |-- postcode_3: string (nullable = true)
 |-- easting: string (nullable = true)
 |-- northing: string (nullable = true)
 |-- positional_quality: string (nullable = true)
 |-- local_authority_name: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- spatial_accuracy: string (nullable = true)
 |-- last_uploaded: string (nullable = true)
 |-- location: string (nullable = true)
 |-- socrata_id: string (nullable = true)



In [52]:
post_code_df

DataFrame[Postcode 1: string, Postcode 2: string, Postcode 3: string, Easting: string, Northing: string, Positional Quality: string, Local Authority Name: string, Longitude: string, Latitude: string, Spatial Accuracy: string, Last Uploaded: string, Location: string, Socrata ID: string]

In [58]:
post_code_df=post_code_df\
.transform(with_std_column_names())\
.withColumn("post_code_geo", f.expr("""CONCAT("SRID=4326;POINT(", latitude," ",longitude,")") """)) \
.selectExpr(
    "postcode_1",
    "postcode_2",
    "postcode_3",
    "local_authority_name",
    "post_code_geo"
)

In [3]:
from utils.spark_utils import get_local_spark_session
from utils.spark_utils import cleanup_col_name, with_std_column_names

spark = get_local_spark_session()

In [36]:
df = spark.read.format('csv').option("header", "true").load("/mnt/c/users/lohith/Downloads/kentPolice/2021-12/2021-12-kent-street.csv")
df = df.transform(with_std_column_names())

In [ ]:
schema = df.schema
schema

StructType(List(StructField(Crime ID,StringType,true),StructField(Month,StringType,true),StructField(Reported by,StringType,true),StructField(Falls within,StringType,true),StructField(Longitude,StringType,true),StructField(Latitude,StringType,true),StructField(Location,StringType,true),StructField(LSOA code,StringType,true),StructField(LSOA name,StringType,true),StructField(Crime type,StringType,true),StructField(Last outcome category,StringType,true),StructField(Context,StringType,true)))

In [37]:
df = spark.read.format('csv').option("header", "true").load("/mnt/c/users/lohith/Downloads/AllCrime/2019-03")


In [42]:
from pyspark.sql import functions as f

# https://www.calculator.net/distance-calculator.html?type=3&la1=53.534436&lo1=-2.239941&la2=53.536556&lo2=-2.23603&ctype=dec&lad1=38&lam1=53&las1=51.36&lau1=n&lod1=77&lom1=2&los1=11.76&lou1=w&lad2=39&lam2=56&las2=58.56&lau2=n&lod2=75&lom2=9&los2=1.08&lou2=w&x=71&y=23#latlog

crime_df = df\
.transform(with_std_column_names())\
.withColumn("log", f.expr("CAST(longitude AS DOUBLE)"))\
.withColumn("lat", f.expr("CAST(latitude AS DOUBLE)"))\
.withColumn("point_geo", f.expr("""CONCAT("SRID=4326;POINT(", latitude," ",longitude,")") """)) \
.selectExpr(
    "crime_id",
    "month",
    "reported_by",
    "crime_type",
    "log",
    "lat",
    "point_geo"
)

In [33]:
df.coalesce(1).write.format('parquet').mode("overwrite").save("/mnt/c/users/lohith/Downloads/AllCrime/201903_parquet")
# 573,111

In [1]:
from pyspark.sql import SparkSession

PROJECT_ROOT = '/home/basal/excelanalysis'
SPARK_NUM_PARTITIONS = 8

DEUTILS_PATH = str(PROJECT_ROOT) + '/resources/DqProfiler-1.0-SNAPSHOT-jar-with-dependencies.jar'
POSTGRES_JDBC = str(PROJECT_ROOT) + '/resources/postgresql-42.3.3.jar'

spark = SparkSession \
.builder \
.appName(f"someThing") \
.config("spark.sql.shuffle.partitions", f"{SPARK_NUM_PARTITIONS}") \
.config('spark.jars', f"{DEUTILS_PATH},{POSTGRES_JDBC}") \
.config('spark.sql.session.timeZone', 'UTC') \
.config('spark.sql.execution.arrow.pyspark.enabled', True) \
.config('spark.debug.maxToStringFields', 0) \
.getOrCreate()


22/02/22 15:54:27 WARN Utils: Your hostname, DESKTOP-9QGN9QQ resolves to a loopback address: 127.0.1.1; using 172.24.240.189 instead (on interface eth0)
22/02/22 15:54:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/02/22 15:54:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
spark.conf.get("spark.jars")

'/home/basal/excelanalysis/resources/DqProfiler-1.0-SNAPSHOT-jar-with-dependencies.jar,/home/basal/excelanalysis/resources/postgresql-42.3.3.jar'

In [ ]:
properties = {
    "user": "postgres", 
    "password": "postgres5432",
    "driver": "org.postgresql.Driver"
  }

In [2]:
jdbcDF = spark.read \
    .format("jdbc") \
    .option("driver", "org.postgresql.Driver") \
    .option("url", "jdbc:postgresql://localhost:5432/osdata") \
    .option("dbtable", "airports") \
    .option("user", "postgres") \
    .option("password", "post5432") \
    .load()

In [43]:
crime_df.write \
    .mode("append") \
    .format("jdbc") \
    .option("SaveMode", "overwrite") \
    .option("driver", "org.postgresql.Driver") \
    .option("url", "jdbc:postgresql://localhost:5432/postgis_db") \
    .option("dbtable", "uk_crime_201903") \
    .option("user", "postgres") \
    .option("password", "post5432") \
    .save()

In [59]:
post_code_df.write \
    .mode("append") \
    .format("jdbc") \
    .option("SaveMode", "overwrite") \
    .option("driver", "org.postgresql.Driver") \
    .option("url", "jdbc:postgresql://localhost:5432/postgis_db") \
    .option("dbtable", "post_code") \
    .option("user", "postgres") \
    .option("password", "post5432") \
    .save()

In [60]:
post_code_df.count()

1770315